In [ ]:
# python main.py --use_labels=False --use_reconst_loss=True
import os
from solver import Solver
from torch.backends import cudnn
from data_loader import get_loader

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
cudnn.benchmark = True 
 
config1 = {
'image_size' : 200, 
'g_conv_dim' : 64, 
'd_conv_dim' : 64, 
'num_classes' : 10, 
'train_iters' : 40000,
'batch_size' : 4,
'num_workers' : 2,
'lr' : 0.0002,
'beta1' : 0.5,
'beta2' : 0.999,

'mode'  :'train',
'model_path' :'./models',
'sample_path' :'./samples',
'mnist_path' :'./mnist',
'svhn_path' :'./svhn',
'log_step' : 10,
'sample_step' : 500,

'use_labels': False, 
'use_reconst_loss' : True, 
}

config = dotdict(config1)
old_loader, young_loader = get_loader(config)

solver = Solver(config, old_loader, young_loader)
solver.train()




In [ ]:
import joblib
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import torchvision
from torch.backends import cudnn
from data_loader import get_loader
from data_utils import FaceData
from model import G12, G21
import numpy as np

%load_ext autoreload
%autoreload 2


cudnn.benchmark = True 
 
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
config1 = {
'image_size' : 200, 
'g_conv_dim' : 64, 
'd_conv_dim' : 64, 
'num_classes' : 10, 
'train_iters' : 40000,
'batch_size' : 4,
'num_workers' : 2,
'lr' : 0.0002,
'beta1' : 0.5,
'beta2' : 0.999,

'mode'  :'train',
'model_path' :'./models',
'sample_path' :'./samples',
'mnist_path' :'./mnist',
'svhn_path' :'./svhn',
'log_step' : 10,
'sample_step' : 500,

'use_labels': False, 
'use_reconst_loss' : True, 
}

config = dotdict(config1)

old = FaceData(image_paths_file='LAG/train/train.txt', young=False)
young = FaceData(image_paths_file='LAG/train/train.txt')

g12 = G12(conv_dim=config.g_conv_dim)
state_G12 = torch.load('models/g12-40000.pkl')
g12.load_state_dict(state_G12)

g21 = G21(conv_dim=config.g_conv_dim)
state_G21 = torch.load('models/g21-40000.pkl')
g21.load_state_dict(state_G21)


num_example_imgs = 3
plt.switch_backend('agg')
plt.ion()

plt.figure(figsize=(15,5))

#for i, (img, img_label) in enumerate(young[-num_example_imgs:]):

img, _ = young[-1]
inputs = img.unsqueeze(0)
inputs = Variable(inputs)
    
outputs = g12.forward(inputs)
reverse = g21.forward(outputs)    
    
pred = outputs.data.cpu()
rev = reverse.data.cpu()

#pred = pred.type(torch.LongTensor)    
img, pred = img.numpy(), pred.numpy()
rev = rev.numpy()
pred = np.around(pred,decimals=2)
pred = pred - np.min(pred)
print((pred - np.min(pred)))

rev = np.around(rev,decimals=2)
rev = rev - np.min(rev)

# img
plt.subplot(1, 2, 1)
plt.axis('off')
print(img.transpose(1,2,0).shape)
plt.imshow(img.transpose(1,2,0))
    
# pred
plt.subplot(1, 2 ,2)
plt.axis('off')
#plt.imshow(pred.reshape(200,200,3))
plt.imshow(rev.reshape(200,200,3))
        
plt.savefig('randomfig.png')
plt.show()


In [2]:
import joblib
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import torchvision
from torch.backends import cudnn
from data_loader import get_loader
from data_utils import FaceData
from model import G12, G21
import numpy as np

%load_ext autoreload
%autoreload 2


cudnn.benchmark = True 
 
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
config1 = {
'image_size' : 200, 
'g_conv_dim' : 64, 
'd_conv_dim' : 64, 
'num_classes' : 10, 
'train_iters' : 40000,
'batch_size' : 4,
'num_workers' : 2,
'lr' : 0.0002,
'beta1' : 0.5,
'beta2' : 0.999,

'mode'  :'train',
'model_path' :'./models',
'sample_path' :'./samples',
'mnist_path' :'./mnist',
'svhn_path' :'./svhn',
'log_step' : 10,
'sample_step' : 500,

'use_labels': False, 
'use_reconst_loss' : True, 
}

config = dotdict(config1)

old = FaceData(image_paths_file='LAG/train/train.txt', young=False)
young = FaceData(image_paths_file='LAG/train/train.txt')

g12 = G12(conv_dim=config.g_conv_dim)
state_G12 = torch.load('models/g12-40000.pkl')
g12.load_state_dict(state_G12)

g21 = G21(conv_dim=config.g_conv_dim)
state_G21 = torch.load('models/g21-40000.pkl')
g21.load_state_dict(state_G21)


plt.switch_backend('agg')
plt.ion()

plt.figure(figsize=(15,5))

num_example_imgs = 5
for i, (img, img_label) in enumerate(young[-num_example_imgs:]):

    inputs = img.unsqueeze(0)
    inputs = Variable(inputs)
    
    outputs = g12(inputs)
    reverse = g21(outputs)    
    
    pred = outputs[0].data.cpu()
    rev = reverse[0].data.cpu()

    img, pred = img.numpy(), pred.numpy()
    rev = rev.numpy()
    print(pred)

    #pred = np.around(pred,decimals=2)
    pred = (pred - np.min(pred))/np.max(pred)*255
    #print((pred - np.min(pred)))

    rev = np.around(rev,decimals=2)
    rev = rev - np.min(rev)

    # img
    plt.subplot(num_example_imgs, 3, i * 3 + 1)
    plt.axis('off')
    print(img.transpose(1,2,0).shape)
    plt.imshow(img.transpose(1,2,0))
    
    # pred
    plt.subplot(num_example_imgs, 3, i * 3 + 2)
    plt.axis('off')
    #plt.imshow(pred.reshape(200,200,3))
    plt.imshow(pred.reshape(200,200,3))
    
    # rev
    plt.subplot(num_example_imgs, 3, i * 3 + 3)
    plt.axis('off')
    #plt.imshow(pred.reshape(200,200,3))
    plt.imshow(rev.reshape(200,200,3))
    
    plt.savefig('randomfig.png')
    plt.show()


[[[ 1.          1.          1.         ...,  1.          1.          1.        ]
  [ 1.          1.          1.         ...,  1.          1.          1.        ]
  [ 1.          1.          1.         ...,  1.          1.          1.        ]
  ..., 
  [ 1.          1.          1.         ...,  1.          1.          0.99999994]
  [ 1.          1.          1.         ...,  1.          1.          1.        ]
  [ 1.          1.          1.         ...,  1.          1.          0.99999994]]

 [[ 1.          1.          1.         ...,  1.          1.          1.        ]
  [ 1.          1.          1.         ...,  1.          1.          1.        ]
  [ 1.          1.          1.         ...,  1.          1.          1.        ]
  ..., 
  [ 1.          1.          1.         ...,  1.          1.          1.        ]
  [ 1.          1.          1.         ...,  1.          1.          1.        ]
  [ 1.          1.          1.         ...,  1.          1.          0.99999994]]

 [[ 1.  